In [1]:
import numpy as np
import healpy as hp
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from astropy.table import Table, join

import os
import time
import healpy as hp
import numpy as np
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt
import numexpr as ne
import math
from scipy.optimize import brentq, newton

In [2]:
def theta_s2g(theta_deg, ra_deg, dec_deg, *, check=False, atol=1e-12, raise_on_fail=False, label="theta_s2g"):
    """
    θ, RA, Dec in degrees.
    Convention: θ = 0° points to west (= -α-hat) and increases counter-clockwise.
    Returns n, g with shape (N, 3).
    """
    # Input -> 1D broadcasted arrays
    ra  = np.radians(np.asarray(ra_deg,  float))
    dec = np.radians(np.asarray(dec_deg, float))
    th  = np.radians(np.asarray(theta_deg, float))
    ra, dec, th = np.broadcast_arrays(ra, dec, th)
    ra = ra.ravel(); dec = dec.ravel(); th = th.ravel()
    N = ra.size

    # n via ang2vec (may come as (3, N))
    n = hp.ang2vec(np.degrees(ra), np.degrees(dec), lonlat=True)  # ang2vec expects degrees
    n = np.asarray(n)
    if n.ndim == 1 and n.size == 3:
        n = n[None, :]                   # (1,3)
    elif n.shape == (3, N):
        n = n.T                          # (N,3)
    elif n.shape != (N, 3):
        raise ValueError(f"Unexpected shape for n: {n.shape} (expected (N,3))")

    # Tangent basis: α-hat (east) and δ-hat (north)
    alpha = np.column_stack((-np.sin(ra),  np.cos(ra),  np.zeros_like(ra)))
    delta = np.column_stack((-np.sin(dec)*np.cos(ra), -np.sin(dec)*np.sin(ra), np.cos(dec)))

    # g: θ measured from west (-alpha), counter-clockwise
    g = (np.sin(th)[:,None] * delta) - (np.cos(th)[:,None] * alpha)

    if check:
        # n·g -> ~0 ; ||g|| -> ~1
        dot = np.einsum('ij,ij->i', n, g)
        norms = np.linalg.norm(g, axis=1)
        max_abs_dot  = float(np.nanmax(np.abs(dot)))
        max_norm_dev = float(np.nanmax(np.abs(norms - 1.0)))
        ok_all = (max_abs_dot <= atol) and (max_norm_dev <= atol)
        print(f"[{label}] max|n·g|={max_abs_dot:.3e}  max||g|-1|={max_norm_dev:.3e}  -> {'OK' if ok_all else 'FAIL'} (atol={atol:g})")
        if raise_on_fail and not ok_all:
            raise AssertionError(f"{label}: checks failed "
                                 f"(max|n·g|={max_abs_dot:.3e}, max||g|-1|={max_norm_dev:.3e}, atol={atol:g})")

    return n, g

def _ensure_tangent_unit(a, n, eps=1e-12):
    """
    Project `a` onto the tangent plane of `n` and normalize.
    If the projection almost vanishes (degenerate), build a generic tangent vector.
    """
    a = np.asarray(a, float)
    n = np.asarray(n, float)
    # Projection onto the tangent plane
    a_t = a - (np.einsum('ij,ij->i', a, n))[:, None] * n
    norms = np.linalg.norm(a_t, axis=1)
    ok = norms > eps
    if not np.all(ok):
        # Build a tangent basis from an axis that is not colinear with n
        e = np.zeros_like(n); e[:,0] = 1.0
        near_x = np.abs(np.einsum('ij,ij->i', n, e)) > 0.9
        e[near_x] = np.array([0.0, 1.0, 0.0])  # switch axis when n ~ x-hat
        # Generic tangent vector: t = n × e
        t = np.cross(n[~ok], e[~ok])
        t_norm = np.linalg.norm(t, axis=1); t_norm[t_norm==0] = 1.0
        t /= t_norm[:, None]
        a_t[~ok] = t
        norms[~ok] = 1.0
    a_t /= norms[:, None]
    return a_t

def _sanitize_sigma_rad(sigma):
    """
    Ensure valid radians with a floor of 1°.
    """
    sigma = np.asarray(sigma, float).copy()
    floor = np.deg2rad(1.0)
    bad = ~np.isfinite(sigma) | (sigma <= 0)
    sigma[bad] = floor
    sigma = np.maximum(sigma, floor)
    return sigma

def save_observed_catalogs(n_gal, a_obs, a_psf, sigma_obs,
                           out_dir="obs_data",
                           obs_name="catalog_obs",
                           psf_name="catalog_obs_psf"):
    """
    Write two FITS files compatible with the estimator:
      - {out_dir}/{obs_name}.fits      (observed)
      - {out_dir}/{psf_name}.fits      (PSF template)
    """
    os.makedirs(out_dir, exist_ok=True)

    n_gal = np.asarray(n_gal, float)
    a_obs = np.asarray(a_obs, float)
    a_psf = np.asarray(a_psf, float)
    sigma_obs = _sanitize_sigma_rad(sigma_obs)

    assert n_gal.ndim == 2 and n_gal.shape[1] == 3, "n_gal must be (N,3)"
    assert a_obs.shape == n_gal.shape, "a_obs must have shape (N,3)"
    assert a_psf.shape == n_gal.shape, "a_psf must have shape (N,3)"
    assert sigma_obs.shape[0] == n_gal.shape[0], "sigma_obs must have N elements (radians)"

    # Ensure that the `a` vectors lie in the tangent plane to `n` and are normalized
    a_obs_t = _ensure_tangent_unit(a_obs, n_gal)
    a_psf_t = _ensure_tangent_unit(a_psf, n_gal)

    # Quick sanity reports
    dot_obs = np.mean(np.abs(np.einsum('ij,ij->i', a_obs_t, n_gal)))
    dot_psf = np.mean(np.abs(np.einsum('ij,ij->i', a_psf_t, n_gal)))
    print(f"[CHECK] <|a_obs·n|> ~ {dot_obs:.3e}, <|a_psf·n|> ~ {dot_psf:.3e}")

    # Observed
    tab_obs = Table(
        [ n_gal[:,0], n_gal[:,1], n_gal[:,2],
          a_obs_t[:,0], a_obs_t[:,1], a_obs_t[:,2],
          sigma_obs ],
        names=("n_x","n_y","n_z","g_rot_x","g_rot_y","g_rot_z","sigma")
    )
    path_obs = os.path.join(out_dir, f"{obs_name}.fits")
    tab_obs.write(path_obs, overwrite=True)
    print(f"[WRITE] {path_obs}  (N={len(tab_obs)})")

    # PSF
    tab_psf = Table(
        [ n_gal[:,0], n_gal[:,1], n_gal[:,2],
          a_psf_t[:,0], a_psf_t[:,1], a_psf_t[:,2],
          sigma_obs ],  # uses the same σ as the observed catalog (per-galaxy measurement)
        names=("n_x","n_y","n_z","g_rot_x","g_rot_y","g_rot_z","sigma")
    )
    path_psf = os.path.join(out_dir, f"{psf_name}.fits")
    tab_psf.write(path_psf, overwrite=True)
    print(f"[WRITE] {path_psf}  (N={len(tab_psf)})")

    # Small extra header (optional): flag as observed
    for path in (path_obs, path_psf):
        with fits.open(path, mode="update") as hdul:
            hdr = hdul[1].header
            hdr["DATAKIND"] = ("OBS", "Observed catalog (not simulation)")
            hdr["UNITSIG"]  = ("RAD", "Position angle uncertainty (radians)")
            hdul.flush()

    return path_obs, path_psf

def floor_sigma(sigma, *, units='rad', min_deg=1.0):
    """
    Return sigma in radians, with a minimum floor = min_deg.
    - units: 'rad' or 'deg' (unit of the input sigma)
    - NaNs, infs and values <= 0 are set to the floor.
    """
    sig = np.asarray(sigma, float)
    if units.lower().startswith('deg'):
        sig = np.radians(sig)
    piso = np.deg2rad(min_deg)
    sig = np.where(~np.isfinite(sig) | (sig <= 0.0), piso, sig)  # sanitize
    sig = np.maximum(sig, piso)                                  # apply floor
    return sig

merged_bulge_obs = Table.read("/home/pedro/ssd2/pcloud/des_ia_legacy/6_classification_and_final_quality_cuts/bulge_tcheng_vegaferrero_sample_binned_hdls.fits")
merged_disc_obs = Table.read("/home/pedro/ssd2/pcloud/des_ia_legacy/6_classification_and_final_quality_cuts/disc_tcheng_vegaferrero_sample_binned_hdls.fits")

n_gal,a_obs = theta_s2g(merged_bulge_obs['theta_s_hdls'],merged_bulge_obs['alphawin_j2000'],merged_bulge_obs['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_bulge_obs['theta_s_psf_hdls'],merged_bulge_obs['alphawin_j2000'],merged_bulge_obs['deltawin_j2000'])
sigma_obs = floor_sigma(merged_bulge_obs['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# assuming you already have n_gal, a_obs, a_psf, sigma_obs (radians)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd",
    obs_name="catalog_obs",
    psf_name="catalog_obs_psf"
)

n_gal,a_obs = theta_s2g(merged_disc_obs['theta_s_hdls'],merged_disc_obs['alphawin_j2000'],merged_disc_obs['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_disc_obs['theta_s_psf_hdls'],merged_disc_obs['alphawin_j2000'],merged_disc_obs['deltawin_j2000'])
sigma_obs = floor_sigma(merged_disc_obs['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# assuming you already have n_gal, a_obs, a_psf, sigma_obs (radians)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd",
    obs_name="catalog_obs",
    psf_name="catalog_obs_psf"
)


[CHECK] <|a_obs·n|> ~ 2.327e-17, <|a_psf·n|> ~ 2.304e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs.fits  (N=1381049)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_psf.fits  (N=1381049)
[CHECK] <|a_obs·n|> ~ 2.328e-17, <|a_psf·n|> ~ 2.305e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs.fits  (N=3007638)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs_psf.fits  (N=3007638)


In [11]:
def theta_s2g(theta_deg, ra_deg, dec_deg, *, check=False, atol=1e-12, raise_on_fail=False, label="theta_s2g"):
    """
    θ, RA, Dec em graus. Convenção: θ=0° aponta para oeste (= -α-hat) e cresce anti-horário.
    Retorna n, g com shape (N,3). (Corrige o shape de n vindo do ang2vec.)
    """
    # Entrada -> arrays 1D broadcastados
    ra  = np.radians(np.asarray(ra_deg,  float))
    dec = np.radians(np.asarray(dec_deg, float))
    th  = np.radians(np.asarray(theta_deg, float))
    ra, dec, th = np.broadcast_arrays(ra, dec, th)
    ra = ra.ravel(); dec = dec.ravel(); th = th.ravel()
    N = ra.size

    # n via ang2vec (pode vir (3,N))
    n = hp.ang2vec(np.degrees(ra), np.degrees(dec), lonlat=True)  # ang2vec espera graus
    n = np.asarray(n)
    if n.ndim == 1 and n.size == 3:
        n = n[None, :]                   # (1,3)
    elif n.shape == (3, N):
        n = n.T                          # (N,3)
    elif n.shape != (N, 3):
        raise ValueError(f"Shape inesperado de n: {n.shape} (esperado (N,3))")

    # Base tangente: α-hat (leste) e δ-hat (norte)
    alpha = np.column_stack((-np.sin(ra),  np.cos(ra),  np.zeros_like(ra)))
    delta = np.column_stack((-np.sin(dec)*np.cos(ra), -np.sin(dec)*np.sin(ra), np.cos(dec)))

    # g: θ medido a partir do oeste (-alpha), anti-horário
    g = (np.sin(th)[:,None] * delta) - (np.cos(th)[:,None] * alpha)

    if check:
        # n·g -> ~0 ; ||g|| -> ~1
        dot = np.einsum('ij,ij->i', n, g)
        norms = np.linalg.norm(g, axis=1)
        max_abs_dot  = float(np.nanmax(np.abs(dot)))
        max_norm_dev = float(np.nanmax(np.abs(norms - 1.0)))
        ok_all = (max_abs_dot <= atol) and (max_norm_dev <= atol)
        print(f"[{label}] max|n·g|={max_abs_dot:.3e}  max||g|-1|={max_norm_dev:.3e}  -> {'OK' if ok_all else 'FAIL'} (atol={atol:g})")
        if raise_on_fail and not ok_all:
            raise AssertionError(f"{label}: checks failed "
                                 f"(max|n·g|={max_abs_dot:.3e}, max||g|-1|={max_norm_dev:.3e}, atol={atol:g})")

    return n, g

def _ensure_tangent_unit(a, n, eps=1e-12):
    """
    Projeta 'a' no plano tangente de 'n' e normaliza.
    Se a projeção quase zera (degenerado), constrói um vetor tangente genérico.
    """
    a = np.asarray(a, float)
    n = np.asarray(n, float)
    # projeção no plano tangente
    a_t = a - (np.einsum('ij,ij->i', a, n))[:, None] * n
    norms = np.linalg.norm(a_t, axis=1)
    ok = norms > eps
    if not np.all(ok):
        # constrói base tangente a partir de um eixo que não seja colinear com n
        e = np.zeros_like(n); e[:,0] = 1.0
        near_x = np.abs(np.einsum('ij,ij->i', n, e)) > 0.9
        e[near_x] = np.array([0.0, 1.0, 0.0])  # troca eixo quando n ~ x-hat
        # vetor tangente genérico: t = n × e
        t = np.cross(n[~ok], e[~ok])
        t_norm = np.linalg.norm(t, axis=1); t_norm[t_norm==0] = 1.0
        t /= t_norm[:, None]
        a_t[~ok] = t
        norms[~ok] = 1.0
    a_t /= norms[:, None]
    return a_t

def _sanitize_sigma_rad(sigma):
    """
    Garante radianos válidos com piso de 1°.
    """
    sigma = np.asarray(sigma, float).copy()
    floor = np.deg2rad(1.0)
    bad = ~np.isfinite(sigma) | (sigma <= 0)
    sigma[bad] = floor
    sigma = np.maximum(sigma, floor)
    return sigma

def save_observed_catalogs(n_gal, a_obs, a_psf, sigma_obs,
                           out_dir="obs_data",
                           obs_name="catalog_obs",
                           psf_name="catalog_obs_psf"):
    """
    Grava dois FITS compatíveis com o estimador:
      - {out_dir}/{obs_name}.fits      (observado)
      - {out_dir}/{psf_name}.fits      (template PSF)
    """
    os.makedirs(out_dir, exist_ok=True)

    n_gal = np.asarray(n_gal, float)
    a_obs = np.asarray(a_obs, float)
    a_psf = np.asarray(a_psf, float)
    sigma_obs = _sanitize_sigma_rad(sigma_obs)

    assert n_gal.ndim == 2 and n_gal.shape[1] == 3, "n_gal deve ser (N,3)"
    assert a_obs.shape == n_gal.shape, "a_obs deve ter shape (N,3)"
    assert a_psf.shape == n_gal.shape, "a_psf deve ter shape (N,3)"
    assert sigma_obs.shape[0] == n_gal.shape[0], "sigma_obs deve ter N elementos (radianos)"

    # Garante que os vetores 'a' estão no plano tangente a 'n' e normalizados
    a_obs_t = _ensure_tangent_unit(a_obs, n_gal)
    a_psf_t = _ensure_tangent_unit(a_psf, n_gal)

    # Relatórios rápidos de sanidade
    dot_obs = np.mean(np.abs(np.einsum('ij,ij->i', a_obs_t, n_gal)))
    dot_psf = np.mean(np.abs(np.einsum('ij,ij->i', a_psf_t, n_gal)))
    print(f"[CHECK] <|a_obs·n|> ~ {dot_obs:.3e}, <|a_psf·n|> ~ {dot_psf:.3e}")

    # Observado
    tab_obs = Table(
        [ n_gal[:,0], n_gal[:,1], n_gal[:,2],
          a_obs_t[:,0], a_obs_t[:,1], a_obs_t[:,2],
          sigma_obs ],
        names=("n_x","n_y","n_z","g_rot_x","g_rot_y","g_rot_z","sigma")
    )
    path_obs = os.path.join(out_dir, f"{obs_name}.fits")
    tab_obs.write(path_obs, overwrite=True)
    print(f"[WRITE] {path_obs}  (N={len(tab_obs)})")

    # PSF
    tab_psf = Table(
        [ n_gal[:,0], n_gal[:,1], n_gal[:,2],
          a_psf_t[:,0], a_psf_t[:,1], a_psf_t[:,2],
          sigma_obs ],  # usa o mesmo σ do observado (medida por galáxia)
        names=("n_x","n_y","n_z","g_rot_x","g_rot_y","g_rot_z","sigma")
    )
    path_psf = os.path.join(out_dir, f"{psf_name}.fits")
    tab_psf.write(path_psf, overwrite=True)
    print(f"[WRITE] {path_psf}  (N={len(tab_psf)})")

    # Pequeno header extra (opcional): sinaliza que é observado
    for path in (path_obs, path_psf):
        with fits.open(path, mode="update") as hdul:
            hdr = hdul[1].header
            hdr["DATAKIND"] = ("OBS", "Observed catalog (not simulation)")
            hdr["UNITSIG"]  = ("RAD", "Position angle uncertainty (radians)")
            hdul.flush()

    return path_obs, path_psf

def floor_sigma(sigma, *, units='rad', min_deg=1.0):
    """
    Devolve sigma em radianos, com piso mínimo = min_deg.
    - units: 'rad' ou 'deg' (unidade do sigma de entrada)
    - NaNs, infs e valores <=0 viram o piso.
    """
    sig = np.asarray(sigma, float)
    if units.lower().startswith('deg'):
        sig = np.radians(sig)
    piso = np.deg2rad(min_deg)
    sig = np.where(~np.isfinite(sig) | (sig <= 0.0), piso, sig)  # saneia
    sig = np.maximum(sig, piso)                                  # aplica piso
    return sig

merged_bulge_obs = Table.read("/home/pedro/ssd2/pcloud/des_ia_legacy/6_classification_and_final_quality_cuts/bulge_tcheng_vegaferrero_sample_binned_hdls.fits")
merged_disc_obs = Table.read("/home/pedro/ssd2/pcloud/des_ia_legacy/6_classification_and_final_quality_cuts/disc_tcheng_vegaferrero_sample_binned_hdls.fits")

bd_z_mask_leq04 = merged_bulge_obs['photo_z']<=0.4
bd_z_mask_g04 = merged_bulge_obs['photo_z']>0.4

dd_z_mask_leq04 = merged_disc_obs['photo_z']<=0.4
dd_z_mask_g04 = merged_disc_obs['photo_z']>0.4

merged_bulge_obs_leq04 = merged_bulge_obs[bd_z_mask_leq04]
merged_bulge_obs_g04 = merged_bulge_obs[bd_z_mask_g04]
merged_disc_obs_leq04 = merged_disc_obs[dd_z_mask_leq04]
merged_disc_obs_g04 = merged_disc_obs[dd_z_mask_g04]

n_gal,a_obs = theta_s2g(merged_bulge_obs_leq04['theta_s_hdls'],merged_bulge_obs_leq04['alphawin_j2000'],merged_bulge_obs_leq04['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_bulge_obs_leq04['theta_s_psf_hdls'],merged_bulge_obs_leq04['alphawin_j2000'],merged_bulge_obs_leq04['deltawin_j2000'])
sigma_obs = floor_sigma(merged_bulge_obs_leq04['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd",
    obs_name="catalog_obs_leq04",
    psf_name="catalog_obs_psf_leq04"
)

n_gal,a_obs = theta_s2g(merged_disc_obs_leq04['theta_s_hdls'],merged_disc_obs_leq04['alphawin_j2000'],merged_disc_obs_leq04['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_disc_obs_leq04['theta_s_psf_hdls'],merged_disc_obs_leq04['alphawin_j2000'],merged_disc_obs_leq04['deltawin_j2000'])
sigma_obs = floor_sigma(merged_disc_obs_leq04['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd",
    obs_name="catalog_obs_leq04",
    psf_name="catalog_obs_psf_leq04"
)

n_gal,a_obs = theta_s2g(merged_bulge_obs_g04['theta_s_hdls'],merged_bulge_obs_g04['alphawin_j2000'],merged_bulge_obs_g04['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_bulge_obs_g04['theta_s_psf_hdls'],merged_bulge_obs_g04['alphawin_j2000'],merged_bulge_obs_g04['deltawin_j2000'])
sigma_obs = floor_sigma(merged_bulge_obs_g04['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd",
    obs_name="catalog_obs_g04",
    psf_name="catalog_obs_psf_g04"
)

n_gal,a_obs = theta_s2g(merged_disc_obs_g04['theta_s_hdls'],merged_disc_obs_g04['alphawin_j2000'],merged_disc_obs_g04['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_disc_obs_g04['theta_s_psf_hdls'],merged_disc_obs_g04['alphawin_j2000'],merged_disc_obs_g04['deltawin_j2000'])
sigma_obs = floor_sigma(merged_disc_obs_g04['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd",
    obs_name="catalog_obs_g04",
    psf_name="catalog_obs_psf_g04"
)



[CHECK] <|a_obs·n|> ~ 2.341e-17, <|a_psf·n|> ~ 2.317e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_leq04.fits  (N=644711)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_psf_leq04.fits  (N=644711)
[CHECK] <|a_obs·n|> ~ 2.322e-17, <|a_psf·n|> ~ 2.302e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs_leq04.fits  (N=1977312)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs_psf_leq04.fits  (N=1977312)
[CHECK] <|a_obs·n|> ~ 2.315e-17, <|a_psf·n|> ~ 2.293e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_g04.fits  (N=736338)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_psf_g04.fits  (N=736338)
[CHECK] <|a_obs·n|> ~ 2.341e-17, <|a_psf·n|> ~ 2.310e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs_g04.fits  (N=1030326)
[WRITE] /home/

In [2]:
def theta_s2g(theta_deg, ra_deg, dec_deg, *, check=False, atol=1e-12, raise_on_fail=False, label="theta_s2g"):
    """
    θ, RA, Dec em graus. Convenção: θ=0° aponta para oeste (= -α-hat) e cresce anti-horário.
    Retorna n, g com shape (N,3). (Corrige o shape de n vindo do ang2vec.)
    """
    # Entrada -> arrays 1D broadcastados
    ra  = np.radians(np.asarray(ra_deg,  float))
    dec = np.radians(np.asarray(dec_deg, float))
    th  = np.radians(np.asarray(theta_deg, float))
    ra, dec, th = np.broadcast_arrays(ra, dec, th)
    ra = ra.ravel(); dec = dec.ravel(); th = th.ravel()
    N = ra.size

    # n via ang2vec (pode vir (3,N))
    n = hp.ang2vec(np.degrees(ra), np.degrees(dec), lonlat=True)  # ang2vec espera graus
    n = np.asarray(n)
    if n.ndim == 1 and n.size == 3:
        n = n[None, :]                   # (1,3)
    elif n.shape == (3, N):
        n = n.T                          # (N,3)
    elif n.shape != (N, 3):
        raise ValueError(f"Shape inesperado de n: {n.shape} (esperado (N,3))")

    # Base tangente: α-hat (leste) e δ-hat (norte)
    alpha = np.column_stack((-np.sin(ra),  np.cos(ra),  np.zeros_like(ra)))
    delta = np.column_stack((-np.sin(dec)*np.cos(ra), -np.sin(dec)*np.sin(ra), np.cos(dec)))

    # g: θ medido a partir do oeste (-alpha), anti-horário
    g = (np.sin(th)[:,None] * delta) - (np.cos(th)[:,None] * alpha)

    if check:
        # n·g -> ~0 ; ||g|| -> ~1
        dot = np.einsum('ij,ij->i', n, g)
        norms = np.linalg.norm(g, axis=1)
        max_abs_dot  = float(np.nanmax(np.abs(dot)))
        max_norm_dev = float(np.nanmax(np.abs(norms - 1.0)))
        ok_all = (max_abs_dot <= atol) and (max_norm_dev <= atol)
        print(f"[{label}] max|n·g|={max_abs_dot:.3e}  max||g|-1|={max_norm_dev:.3e}  -> {'OK' if ok_all else 'FAIL'} (atol={atol:g})")
        if raise_on_fail and not ok_all:
            raise AssertionError(f"{label}: checks failed "
                                 f"(max|n·g|={max_abs_dot:.3e}, max||g|-1|={max_norm_dev:.3e}, atol={atol:g})")

    return n, g

def _ensure_tangent_unit(a, n, eps=1e-12):
    """
    Projeta 'a' no plano tangente de 'n' e normaliza.
    Se a projeção quase zera (degenerado), constrói um vetor tangente genérico.
    """
    a = np.asarray(a, float)
    n = np.asarray(n, float)
    # projeção no plano tangente
    a_t = a - (np.einsum('ij,ij->i', a, n))[:, None] * n
    norms = np.linalg.norm(a_t, axis=1)
    ok = norms > eps
    if not np.all(ok):
        # constrói base tangente a partir de um eixo que não seja colinear com n
        e = np.zeros_like(n); e[:,0] = 1.0
        near_x = np.abs(np.einsum('ij,ij->i', n, e)) > 0.9
        e[near_x] = np.array([0.0, 1.0, 0.0])  # troca eixo quando n ~ x-hat
        # vetor tangente genérico: t = n × e
        t = np.cross(n[~ok], e[~ok])
        t_norm = np.linalg.norm(t, axis=1); t_norm[t_norm==0] = 1.0
        t /= t_norm[:, None]
        a_t[~ok] = t
        norms[~ok] = 1.0
    a_t /= norms[:, None]
    return a_t

def _sanitize_sigma_rad(sigma):
    """
    Garante radianos válidos com piso de 1°.
    """
    sigma = np.asarray(sigma, float).copy()
    floor = np.deg2rad(1.0)
    bad = ~np.isfinite(sigma) | (sigma <= 0)
    sigma[bad] = floor
    sigma = np.maximum(sigma, floor)
    return sigma

def save_observed_catalogs(n_gal, a_obs, a_psf, sigma_obs,
                           out_dir="obs_data",
                           obs_name="catalog_obs",
                           psf_name="catalog_obs_psf"):
    """
    Grava dois FITS compatíveis com o estimador:
      - {out_dir}/{obs_name}.fits      (observado)
      - {out_dir}/{psf_name}.fits      (template PSF)
    """
    os.makedirs(out_dir, exist_ok=True)

    n_gal = np.asarray(n_gal, float)
    a_obs = np.asarray(a_obs, float)
    a_psf = np.asarray(a_psf, float)
    sigma_obs = _sanitize_sigma_rad(sigma_obs)

    assert n_gal.ndim == 2 and n_gal.shape[1] == 3, "n_gal deve ser (N,3)"
    assert a_obs.shape == n_gal.shape, "a_obs deve ter shape (N,3)"
    assert a_psf.shape == n_gal.shape, "a_psf deve ter shape (N,3)"
    assert sigma_obs.shape[0] == n_gal.shape[0], "sigma_obs deve ter N elementos (radianos)"

    # Garante que os vetores 'a' estão no plano tangente a 'n' e normalizados
    a_obs_t = _ensure_tangent_unit(a_obs, n_gal)
    a_psf_t = _ensure_tangent_unit(a_psf, n_gal)

    # Relatórios rápidos de sanidade
    dot_obs = np.mean(np.abs(np.einsum('ij,ij->i', a_obs_t, n_gal)))
    dot_psf = np.mean(np.abs(np.einsum('ij,ij->i', a_psf_t, n_gal)))
    print(f"[CHECK] <|a_obs·n|> ~ {dot_obs:.3e}, <|a_psf·n|> ~ {dot_psf:.3e}")

    # Observado
    tab_obs = Table(
        [ n_gal[:,0], n_gal[:,1], n_gal[:,2],
          a_obs_t[:,0], a_obs_t[:,1], a_obs_t[:,2],
          sigma_obs ],
        names=("n_x","n_y","n_z","g_rot_x","g_rot_y","g_rot_z","sigma")
    )
    path_obs = os.path.join(out_dir, f"{obs_name}.fits")
    tab_obs.write(path_obs, overwrite=True)
    print(f"[WRITE] {path_obs}  (N={len(tab_obs)})")

    # PSF
    tab_psf = Table(
        [ n_gal[:,0], n_gal[:,1], n_gal[:,2],
          a_psf_t[:,0], a_psf_t[:,1], a_psf_t[:,2],
          sigma_obs ],  # usa o mesmo σ do observado (medida por galáxia)
        names=("n_x","n_y","n_z","g_rot_x","g_rot_y","g_rot_z","sigma")
    )
    path_psf = os.path.join(out_dir, f"{psf_name}.fits")
    tab_psf.write(path_psf, overwrite=True)
    print(f"[WRITE] {path_psf}  (N={len(tab_psf)})")

    # Pequeno header extra (opcional): sinaliza que é observado
    for path in (path_obs, path_psf):
        with fits.open(path, mode="update") as hdul:
            hdr = hdul[1].header
            hdr["DATAKIND"] = ("OBS", "Observed catalog (not simulation)")
            hdr["UNITSIG"]  = ("RAD", "Position angle uncertainty (radians)")
            hdul.flush()

    return path_obs, path_psf

def floor_sigma(sigma, *, units='rad', min_deg=1.0):
    """
    Devolve sigma em radianos, com piso mínimo = min_deg.
    - units: 'rad' ou 'deg' (unidade do sigma de entrada)
    - NaNs, infs e valores <=0 viram o piso.
    """
    sig = np.asarray(sigma, float)
    if units.lower().startswith('deg'):
        sig = np.radians(sig)
    piso = np.deg2rad(min_deg)
    sig = np.where(~np.isfinite(sig) | (sig <= 0.0), piso, sig)  # saneia
    sig = np.maximum(sig, piso)                                  # aplica piso
    return sig

merged_bulge_obs = Table.read("/home/pedro/ssd2/pcloud/des_ia_legacy/6_classification_and_final_quality_cuts/bulge_tcheng_vegaferrero_sample_binned_hdls.fits")
merged_disc_obs = Table.read("/home/pedro/ssd2/pcloud/des_ia_legacy/6_classification_and_final_quality_cuts/disc_tcheng_vegaferrero_sample_binned_hdls.fits")

bd_z_mask_leqDEC35 = merged_bulge_obs['deltawin_j2000']<=-35
bd_z_mask_gDEC35 = merged_bulge_obs['deltawin_j2000']>-35

dd_z_mask_leqDEC35 = merged_disc_obs['deltawin_j2000']<=-35
dd_z_mask_gDEC35 = merged_disc_obs['deltawin_j2000']>-35

merged_bulge_obs_leqDEC35 = merged_bulge_obs[bd_z_mask_leqDEC35]
merged_bulge_obs_gDEC35 = merged_bulge_obs[bd_z_mask_gDEC35]
merged_disc_obs_leqDEC35 = merged_disc_obs[dd_z_mask_leqDEC35]
merged_disc_obs_gDEC35 = merged_disc_obs[dd_z_mask_gDEC35]

n_gal,a_obs = theta_s2g(merged_bulge_obs_leqDEC35['theta_s_hdls'],merged_bulge_obs_leqDEC35['alphawin_j2000'],merged_bulge_obs_leqDEC35['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_bulge_obs_leqDEC35['theta_s_psf_hdls'],merged_bulge_obs_leqDEC35['alphawin_j2000'],merged_bulge_obs_leqDEC35['deltawin_j2000'])
sigma_obs = floor_sigma(merged_bulge_obs_leqDEC35['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd",
    obs_name="catalog_obs_leqDEC35",
    psf_name="catalog_obs_psf_leqDEC35"
)

n_gal,a_obs = theta_s2g(merged_disc_obs_leqDEC35['theta_s_hdls'],merged_disc_obs_leqDEC35['alphawin_j2000'],merged_disc_obs_leqDEC35['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_disc_obs_leqDEC35['theta_s_psf_hdls'],merged_disc_obs_leqDEC35['alphawin_j2000'],merged_disc_obs_leqDEC35['deltawin_j2000'])
sigma_obs = floor_sigma(merged_disc_obs_leqDEC35['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd",
    obs_name="catalog_obs_leqDEC35",
    psf_name="catalog_obs_psf_leqDEC35"
)

n_gal,a_obs = theta_s2g(merged_bulge_obs_gDEC35['theta_s_hdls'],merged_bulge_obs_gDEC35['alphawin_j2000'],merged_bulge_obs_gDEC35['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_bulge_obs_gDEC35['theta_s_psf_hdls'],merged_bulge_obs_gDEC35['alphawin_j2000'],merged_bulge_obs_gDEC35['deltawin_j2000'])
sigma_obs = floor_sigma(merged_bulge_obs_gDEC35['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd",
    obs_name="catalog_obs_gDEC35",
    psf_name="catalog_obs_psf_gDEC35"
)

n_gal,a_obs = theta_s2g(merged_disc_obs_gDEC35['theta_s_hdls'],merged_disc_obs_gDEC35['alphawin_j2000'],merged_disc_obs_gDEC35['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_disc_obs_gDEC35['theta_s_psf_hdls'],merged_disc_obs_gDEC35['alphawin_j2000'],merged_disc_obs_gDEC35['deltawin_j2000'])
sigma_obs = floor_sigma(merged_disc_obs_gDEC35['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd",
    obs_name="catalog_obs_gDEC35",
    psf_name="catalog_obs_psf_gDEC35"
)



[CHECK] <|a_obs·n|> ~ 2.586e-17, <|a_psf·n|> ~ 2.541e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_leqDEC35.fits  (N=682745)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_psf_leqDEC35.fits  (N=682745)
[CHECK] <|a_obs·n|> ~ 2.578e-17, <|a_psf·n|> ~ 2.531e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs_leqDEC35.fits  (N=1538678)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs_psf_leqDEC35.fits  (N=1538678)
[CHECK] <|a_obs·n|> ~ 2.074e-17, <|a_psf·n|> ~ 2.072e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_gDEC35.fits  (N=698304)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_psf_gDEC35.fits  (N=698304)
[CHECK] <|a_obs·n|> ~ 2.067e-17, <|a_psf·n|> ~ 2.068e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs_gDEC35.fits  (N=14

In [ ]:
mask_bulge_gra30 = (merged_bulge_obs['alphawin_j2000']>30) & (merged_bulge_obs['alphawin_j2000']<200)
mask_bulge_leqra30 = ~mask_bulge_gra30

In [2]:
def theta_s2g(theta_deg, ra_deg, dec_deg, *, check=False, atol=1e-12, raise_on_fail=False, label="theta_s2g"):
    """
    θ, RA, Dec em graus. Convenção: θ=0° aponta para oeste (= -α-hat) e cresce anti-horário.
    Retorna n, g com shape (N,3). (Corrige o shape de n vindo do ang2vec.)
    """
    # Entrada -> arrays 1D broadcastados
    ra  = np.radians(np.asarray(ra_deg,  float))
    dec = np.radians(np.asarray(dec_deg, float))
    th  = np.radians(np.asarray(theta_deg, float))
    ra, dec, th = np.broadcast_arrays(ra, dec, th)
    ra = ra.ravel(); dec = dec.ravel(); th = th.ravel()
    N = ra.size

    # n via ang2vec (pode vir (3,N))
    n = hp.ang2vec(np.degrees(ra), np.degrees(dec), lonlat=True)  # ang2vec espera graus
    n = np.asarray(n)
    if n.ndim == 1 and n.size == 3:
        n = n[None, :]                   # (1,3)
    elif n.shape == (3, N):
        n = n.T                          # (N,3)
    elif n.shape != (N, 3):
        raise ValueError(f"Shape inesperado de n: {n.shape} (esperado (N,3))")

    # Base tangente: α-hat (leste) e δ-hat (norte)
    alpha = np.column_stack((-np.sin(ra),  np.cos(ra),  np.zeros_like(ra)))
    delta = np.column_stack((-np.sin(dec)*np.cos(ra), -np.sin(dec)*np.sin(ra), np.cos(dec)))

    # g: θ medido a partir do oeste (-alpha), anti-horário
    g = (np.sin(th)[:,None] * delta) - (np.cos(th)[:,None] * alpha)

    if check:
        # n·g -> ~0 ; ||g|| -> ~1
        dot = np.einsum('ij,ij->i', n, g)
        norms = np.linalg.norm(g, axis=1)
        max_abs_dot  = float(np.nanmax(np.abs(dot)))
        max_norm_dev = float(np.nanmax(np.abs(norms - 1.0)))
        ok_all = (max_abs_dot <= atol) and (max_norm_dev <= atol)
        print(f"[{label}] max|n·g|={max_abs_dot:.3e}  max||g|-1|={max_norm_dev:.3e}  -> {'OK' if ok_all else 'FAIL'} (atol={atol:g})")
        if raise_on_fail and not ok_all:
            raise AssertionError(f"{label}: checks failed "
                                 f"(max|n·g|={max_abs_dot:.3e}, max||g|-1|={max_norm_dev:.3e}, atol={atol:g})")

    return n, g

def _ensure_tangent_unit(a, n, eps=1e-12):
    """
    Projeta 'a' no plano tangente de 'n' e normaliza.
    Se a projeção quase zera (degenerado), constrói um vetor tangente genérico.
    """
    a = np.asarray(a, float)
    n = np.asarray(n, float)
    # projeção no plano tangente
    a_t = a - (np.einsum('ij,ij->i', a, n))[:, None] * n
    norms = np.linalg.norm(a_t, axis=1)
    ok = norms > eps
    if not np.all(ok):
        # constrói base tangente a partir de um eixo que não seja colinear com n
        e = np.zeros_like(n); e[:,0] = 1.0
        near_x = np.abs(np.einsum('ij,ij->i', n, e)) > 0.9
        e[near_x] = np.array([0.0, 1.0, 0.0])  # troca eixo quando n ~ x-hat
        # vetor tangente genérico: t = n × e
        t = np.cross(n[~ok], e[~ok])
        t_norm = np.linalg.norm(t, axis=1); t_norm[t_norm==0] = 1.0
        t /= t_norm[:, None]
        a_t[~ok] = t
        norms[~ok] = 1.0
    a_t /= norms[:, None]
    return a_t

def _sanitize_sigma_rad(sigma):
    """
    Garante radianos válidos com piso de 1°.
    """
    sigma = np.asarray(sigma, float).copy()
    floor = np.deg2rad(1.0)
    bad = ~np.isfinite(sigma) | (sigma <= 0)
    sigma[bad] = floor
    sigma = np.maximum(sigma, floor)
    return sigma

def save_observed_catalogs(n_gal, a_obs, a_psf, sigma_obs,
                           out_dir="obs_data",
                           obs_name="catalog_obs",
                           psf_name="catalog_obs_psf"):
    """
    Grava dois FITS compatíveis com o estimador:
      - {out_dir}/{obs_name}.fits      (observado)
      - {out_dir}/{psf_name}.fits      (template PSF)
    """
    os.makedirs(out_dir, exist_ok=True)

    n_gal = np.asarray(n_gal, float)
    a_obs = np.asarray(a_obs, float)
    a_psf = np.asarray(a_psf, float)
    sigma_obs = _sanitize_sigma_rad(sigma_obs)

    assert n_gal.ndim == 2 and n_gal.shape[1] == 3, "n_gal deve ser (N,3)"
    assert a_obs.shape == n_gal.shape, "a_obs deve ter shape (N,3)"
    assert a_psf.shape == n_gal.shape, "a_psf deve ter shape (N,3)"
    assert sigma_obs.shape[0] == n_gal.shape[0], "sigma_obs deve ter N elementos (radianos)"

    # Garante que os vetores 'a' estão no plano tangente a 'n' e normalizados
    a_obs_t = _ensure_tangent_unit(a_obs, n_gal)
    a_psf_t = _ensure_tangent_unit(a_psf, n_gal)

    # Relatórios rápidos de sanidade
    dot_obs = np.mean(np.abs(np.einsum('ij,ij->i', a_obs_t, n_gal)))
    dot_psf = np.mean(np.abs(np.einsum('ij,ij->i', a_psf_t, n_gal)))
    print(f"[CHECK] <|a_obs·n|> ~ {dot_obs:.3e}, <|a_psf·n|> ~ {dot_psf:.3e}")

    # Observado
    tab_obs = Table(
        [ n_gal[:,0], n_gal[:,1], n_gal[:,2],
          a_obs_t[:,0], a_obs_t[:,1], a_obs_t[:,2],
          sigma_obs ],
        names=("n_x","n_y","n_z","g_rot_x","g_rot_y","g_rot_z","sigma")
    )
    path_obs = os.path.join(out_dir, f"{obs_name}.fits")
    tab_obs.write(path_obs, overwrite=True)
    print(f"[WRITE] {path_obs}  (N={len(tab_obs)})")

    # PSF
    tab_psf = Table(
        [ n_gal[:,0], n_gal[:,1], n_gal[:,2],
          a_psf_t[:,0], a_psf_t[:,1], a_psf_t[:,2],
          sigma_obs ],  # usa o mesmo σ do observado (medida por galáxia)
        names=("n_x","n_y","n_z","g_rot_x","g_rot_y","g_rot_z","sigma")
    )
    path_psf = os.path.join(out_dir, f"{psf_name}.fits")
    tab_psf.write(path_psf, overwrite=True)
    print(f"[WRITE] {path_psf}  (N={len(tab_psf)})")

    # Pequeno header extra (opcional): sinaliza que é observado
    for path in (path_obs, path_psf):
        with fits.open(path, mode="update") as hdul:
            hdr = hdul[1].header
            hdr["DATAKIND"] = ("OBS", "Observed catalog (not simulation)")
            hdr["UNITSIG"]  = ("RAD", "Position angle uncertainty (radians)")
            hdul.flush()

    return path_obs, path_psf

def floor_sigma(sigma, *, units='rad', min_deg=1.0):
    """
    Devolve sigma em radianos, com piso mínimo = min_deg.
    - units: 'rad' ou 'deg' (unidade do sigma de entrada)
    - NaNs, infs e valores <=0 viram o piso.
    """
    sig = np.asarray(sigma, float)
    if units.lower().startswith('deg'):
        sig = np.radians(sig)
    piso = np.deg2rad(min_deg)
    sig = np.where(~np.isfinite(sig) | (sig <= 0.0), piso, sig)  # saneia
    sig = np.maximum(sig, piso)                                  # aplica piso
    return sig

merged_bulge_obs = Table.read("/home/pedro/ssd2/pcloud/des_ia_legacy/6_classification_and_final_quality_cuts/bulge_tcheng_vegaferrero_sample_binned_hdls.fits")
merged_disc_obs = Table.read("/home/pedro/ssd2/pcloud/des_ia_legacy/6_classification_and_final_quality_cuts/disc_tcheng_vegaferrero_sample_binned_hdls.fits")

mask_bulge_gra30 = (merged_bulge_obs['alphawin_j2000']>30) & (merged_bulge_obs['alphawin_j2000']<200)
mask_bulge_leqra30 = ~mask_bulge_gra30

mask_disc_gra30 = (merged_disc_obs['alphawin_j2000']>30) & (merged_disc_obs['alphawin_j2000']<200)
mask_disc_leqra30 = ~mask_disc_gra30

merged_bulge_obs_gRA30 = merged_bulge_obs[mask_bulge_gra30]
merged_bulge_obs_leqRA30 = merged_bulge_obs[mask_bulge_leqra30]
merged_disc_obs_gRA30 = merged_disc_obs[mask_disc_gra30]
merged_disc_obs_leqRA30 = merged_disc_obs[mask_disc_leqra30]

n_gal,a_obs = theta_s2g(merged_bulge_obs_gRA30['theta_s_hdls'],merged_bulge_obs_gRA30['alphawin_j2000'],merged_bulge_obs_gRA30['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_bulge_obs_gRA30['theta_s_psf_hdls'],merged_bulge_obs_gRA30['alphawin_j2000'],merged_bulge_obs_gRA30['deltawin_j2000'])
sigma_obs = floor_sigma(merged_bulge_obs_gRA30['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd",
    obs_name="catalog_obs_gRA30",
    psf_name="catalog_obs_psf_gRA30"
)

n_gal,a_obs = theta_s2g(merged_bulge_obs_leqRA30['theta_s_hdls'],merged_bulge_obs_leqRA30['alphawin_j2000'],merged_bulge_obs_leqRA30['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_bulge_obs_leqRA30['theta_s_psf_hdls'],merged_bulge_obs_leqRA30['alphawin_j2000'],merged_bulge_obs_leqRA30['deltawin_j2000'])
sigma_obs = floor_sigma(merged_bulge_obs_leqRA30['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd",
    obs_name="catalog_obs_leqRA30",
    psf_name="catalog_obs_psf_leqRA30"
)

n_gal,a_obs = theta_s2g(merged_disc_obs_leqRA30['theta_s_hdls'],merged_disc_obs_leqRA30['alphawin_j2000'],merged_disc_obs_leqRA30['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_disc_obs_leqRA30['theta_s_psf_hdls'],merged_disc_obs_leqRA30['alphawin_j2000'],merged_disc_obs_leqRA30['deltawin_j2000'])
sigma_obs = floor_sigma(merged_disc_obs_leqRA30['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd",
    obs_name="catalog_obs_leqRA30",
    psf_name="catalog_obs_psf_leqRA30"
)

n_gal,a_obs = theta_s2g(merged_disc_obs_gRA30['theta_s_hdls'],merged_disc_obs_gRA30['alphawin_j2000'],merged_disc_obs_gRA30['deltawin_j2000'])
n2,a_psf = theta_s2g(merged_disc_obs_gRA30['theta_s_psf_hdls'],merged_disc_obs_gRA30['alphawin_j2000'],merged_disc_obs_gRA30['deltawin_j2000'])
sigma_obs = floor_sigma(merged_disc_obs_gRA30['errtheta_s'], units='rad', min_deg=1) #merged_disc_obs['errtheta_s']

# supondo que você já tenha n_gal, a_obs, a_psf, sigma_obs (radianos)
obs_path, psf_path = save_observed_catalogs(
    n_gal, a_obs, a_psf, sigma_obs,
    out_dir="/home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd",
    obs_name="catalog_obs_gRA30",
    psf_name="catalog_obs_psf_gRA30"
)



[CHECK] <|a_obs·n|> ~ 2.555e-17, <|a_psf·n|> ~ 2.533e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_gRA30.fits  (N=709421)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_psf_gRA30.fits  (N=709421)
[CHECK] <|a_obs·n|> ~ 2.087e-17, <|a_psf·n|> ~ 2.062e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_leqRA30.fits  (N=671628)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_bd/catalog_obs_psf_leqRA30.fits  (N=671628)
[CHECK] <|a_obs·n|> ~ 2.095e-17, <|a_psf·n|> ~ 2.073e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs_leqRA30.fits  (N=1567186)
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs_psf_leqRA30.fits  (N=1567186)
[CHECK] <|a_obs·n|> ~ 2.583e-17, <|a_psf·n|> ~ 2.557e-17
[WRITE] /home/pedro/ssd2/pcloud/des_ia_legacy/new_estimator/obs_data_dd/catalog_obs_gRA30.fits  (N=1440452)
